In [3]:
# First we will import the necessary Library

import os
import pandas as pd
import numpy as np
import math
import datetime as dt

# For Evalution we will use these library

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

# For model building we will use these library

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau



# For PLotting we will use these library

import matplotlib.pyplot as plt
from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
%matplotlib inline
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected=True)

In [4]:
df=pd.read_csv('E:/cryptics/dataset/Potato_Red.csv')
df
closedf = df[['sn','price']]
print("Shape of close dataframe:", closedf.shape)
del closedf['sn']
scaler=MinMaxScaler(feature_range=(0,1))
closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))
print(closedf.shape)

training_size=int(len(closedf)*0.70)
test_size=len(closedf)-training_size
train_data,test_data=closedf[0:training_size,:],closedf[training_size:len(closedf),:1]


def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)
time_step = 30
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)


# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

#red potato
model=Sequential()

model.add(LSTM(128,input_shape=(None,1),activation="relu"))

model.add(Dense(1))

model.compile(loss="mean_squared_error",optimizer="adam")

#early stopping and model checking

#Early Stopping
earlystop = EarlyStopping(monitor = 'val_loss', #value being monitored for improvement
                          min_delta = 0.001,         #Abs value and is the main change required before we stop
                          mode='auto',
                          patience = 6,             #no of epocs we wait before stopping
                          verbose = 1,
                          restore_best_weights = True) #keep the best weigts once stopped
#Model Checkpoint
checkpoint = ModelCheckpoint(monitor = "val_loss",
                             mode ='auto',
                             filepath = "E:/cryptics/dataset/best_model.keras",
                             verbose = 1,
                             save_best_only = True,
                             )

#Reduce learning Rate

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = 0.01,
                              patience = 6,
                              verbose = 1,
                              min_delta = 0.001)


# we put our call backs into a callback list
callback = [earlystop, checkpoint,reduce_lr]

history = model.fit(X_train,y_train,validation_data=(X_test,y_test),callbacks=callback,epochs=220,batch_size=32,verbose=1)

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(loss))



### Lets Do the prediction and check performance metrics
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
train_predict.shape, test_predict.shape

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1))
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1))


# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Train data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

print("Train data explained variance regression score:",
      explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:",
      explained_variance_score(original_ytest, test_predict))


print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

print("Train data MGD: ", mean_gamma_deviance(original_ytrain, train_predict))
print("Test data MGD: ", mean_gamma_deviance(original_ytest, test_predict))
print("----------------------------------------------------------------------")
print("Train data MPD: ", mean_poisson_deviance(original_ytrain, train_predict))
print("Test data MPD: ", mean_poisson_deviance(original_ytest, test_predict))

import seaborn as sns

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)


x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = 30    # Prediction for next 30 days

while(i<pred_days):

    if(len(temp_input)>time_step):

        x_input=np.array(temp_input[1:])
        #print("{} day input {}".format(i,x_input))
        x_input = x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))

        yhat = model.predict(x_input, verbose=0)
        #print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)

        lst_output.extend(yhat.tolist())
        i=i+1

    else:

        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        temp_input.extend(yhat[0].tolist())

        lst_output.extend(yhat.tolist())
        i=i+1

print("Output of predicted next days: ", len(lst_output))

last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
print(last_days)
print(day_pred)

temp_mat = np.empty((len(last_days)+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_days_value = temp_mat
next_predicted_days_value = temp_mat

last_original_days_value[0:time_step+1] = scaler.inverse_transform(closedf[len(closedf)-time_step:]).reshape(1,-1).tolist()[0]
next_predicted_days_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

new_pred_plot = pd.DataFrame({
    'last_original_days_value':last_original_days_value,
    'next_predicted_days_value':next_predicted_days_value
})

names = cycle(['Last 15 days close price','Predicted next 30 days close price'])


# Assuming you have new_pred_plot DataFrame with necessary columns



lstmdf=closedf.tolist()
lstmdf.extend((np.array(lst_output).reshape(-1,1)).tolist())
lstmdf=scaler.inverse_transform(lstmdf).reshape(1,-1).tolist()[0]



# Assuming you have lstmdf list of stock prices





print(next_predicted_days_value[:30])
print(len(next_predicted_days_value))
current =df.iloc[len(df)-1]
yesterday = df.iloc[len(df)-2]

predictedprices=next_predicted_days_value[:30]

Shape of close dataframe: (3595, 2)
(3595, 1)
Epoch 1/220


c:\Users\itssh\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



77/78 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0152
Epoch 1: val_loss improved from inf to 0.00217, saving model to E:/cryptics/dataset/best_model.keras
78/78 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - loss: 0.0150 - val_loss: 0.0022 - learning_rate: 0.0010
Epoch 2/220
76/78 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0013
Epoch 2: val_loss improved from 0.00217 to 0.00188, saving model to E:/cryptics/dataset/best_model.keras
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 0.0013 - val_loss: 0.0019 - learning_rate: 0.0010
Epoch 3/220
76/78 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0011
Epoch 3: val_loss improved from 0.00188 to 0.00107, saving model to E:/cryptics/dataset/best_model.keras
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 0.0011 - val_loss: 0.0011 - learning_rate: 0.0010
Epoch 4/220
76/78 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 8.8906e-04
Epoch 4: val_loss improved from 0.00107 to 0.00095, saving model to E:/cryptics/dataset/best_model.keras
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 

In [5]:
df=pd.read_csv('E:/cryptics/dataset/Banana.csv')
df
closedf = df[['sn','price']]
print("Shape of close dataframe:", closedf.shape)
del closedf['sn']
scaler=MinMaxScaler(feature_range=(0,1))
closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))
print(closedf.shape)

training_size=int(len(closedf)*0.70)
test_size=len(closedf)-training_size
train_data,test_data=closedf[0:training_size,:],closedf[training_size:len(closedf),:1]


def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)
time_step = 30
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)


# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

#red potato
model=Sequential()

model.add(LSTM(128,input_shape=(None,1),activation="relu"))

model.add(Dense(1))

model.compile(loss="mean_squared_error",optimizer="adam")

#early stopping and model checking

#Early Stopping
earlystop = EarlyStopping(monitor = 'val_loss', #value being monitored for improvement
                          min_delta = 0.001,         #Abs value and is the main change required before we stop
                          mode='auto',
                          patience = 6,             #no of epocs we wait before stopping
                          verbose = 1,
                          restore_best_weights = True) #keep the best weigts once stopped
#Model Checkpoint
checkpoint = ModelCheckpoint(monitor = "val_loss",
                             mode ='auto',
                             filepath = "E:/cryptics/dataset/best_model_banana.keras",
                             verbose = 1,
                             save_best_only = True,
                             )

#Reduce learning Rate

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = 0.01,
                              patience = 6,
                              verbose = 1,
                              min_delta = 0.001)


# we put our call backs into a callback list
callback = [earlystop, checkpoint,reduce_lr]

history = model.fit(X_train,y_train,validation_data=(X_test,y_test),callbacks=callback,epochs=220,batch_size=32,verbose=1)

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(loss))



### Lets Do the prediction and check performance metrics
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
train_predict.shape, test_predict.shape

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1))
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1))


# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Train data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

print("Train data explained variance regression score:",
      explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:",
      explained_variance_score(original_ytest, test_predict))


print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

print("Train data MGD: ", mean_gamma_deviance(original_ytrain, train_predict))
print("Test data MGD: ", mean_gamma_deviance(original_ytest, test_predict))
print("----------------------------------------------------------------------")
print("Train data MPD: ", mean_poisson_deviance(original_ytrain, train_predict))
print("Test data MPD: ", mean_poisson_deviance(original_ytest, test_predict))

import seaborn as sns

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)


x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = 30    # Prediction for next 30 days

while(i<pred_days):

    if(len(temp_input)>time_step):

        x_input=np.array(temp_input[1:])
        #print("{} day input {}".format(i,x_input))
        x_input = x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))

        yhat = model.predict(x_input, verbose=0)
        #print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)

        lst_output.extend(yhat.tolist())
        i=i+1

    else:

        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        temp_input.extend(yhat[0].tolist())

        lst_output.extend(yhat.tolist())
        i=i+1

print("Output of predicted next days: ", len(lst_output))

last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
print(last_days)
print(day_pred)

temp_mat = np.empty((len(last_days)+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_days_value = temp_mat
next_predicted_days_value = temp_mat

last_original_days_value[0:time_step+1] = scaler.inverse_transform(closedf[len(closedf)-time_step:]).reshape(1,-1).tolist()[0]
next_predicted_days_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

new_pred_plot = pd.DataFrame({
    'last_original_days_value':last_original_days_value,
    'next_predicted_days_value':next_predicted_days_value
})

names = cycle(['Last 15 days close price','Predicted next 30 days close price'])


# Assuming you have new_pred_plot DataFrame with necessary columns



lstmdf=closedf.tolist()
lstmdf.extend((np.array(lst_output).reshape(-1,1)).tolist())
lstmdf=scaler.inverse_transform(lstmdf).reshape(1,-1).tolist()[0]



# Assuming you have lstmdf list of stock prices





print(next_predicted_days_value[:30])
print(len(next_predicted_days_value))
current1 =df.iloc[len(df)-1]
yesterday1 = df.iloc[len(df)-2]

predictedprices1=next_predicted_days_value[:30]

Shape of close dataframe: (3604, 2)
(3604, 1)
Epoch 1/220


c:\Users\itssh\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0018
Epoch 1: val_loss improved from inf to 0.00083, saving model to E:/cryptics/dataset/best_model_banana.keras
78/78 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - loss: 0.0018 - val_loss: 8.2963e-04 - learning_rate: 0.0010
Epoch 2/220
77/78 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0014
Epoch 2: val_loss did not improve from 0.00083
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0014 - val_loss: 0.0013 - learning_rate: 0.0010
Epoch 3/220
77/78 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0010
Epoch 3: val_loss did not improve from 0.00083
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0010 - val_loss: 0.0013 - learning_rate: 0.0010
Epoch 4/220
75/78 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 8.4400e-04
Epoch 4: val_loss did not improve from 0.00083
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 8.4772e-04 - val_loss: 0.0014 - learning_rate: 0.0010
Epoch 5/220
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0014
Epoch 5: val_loss did no

In [6]:
df=pd.read_csv('E:/cryptics/dataset/Cauli_Local.csv')
df
closedf = df[['sn','price']]
print("Shape of close dataframe:", closedf.shape)
del closedf['sn']
scaler=MinMaxScaler(feature_range=(0,1))
closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))
print(closedf.shape)

training_size=int(len(closedf)*0.70)
test_size=len(closedf)-training_size
train_data,test_data=closedf[0:training_size,:],closedf[training_size:len(closedf),:1]


def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)
time_step = 30
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)


# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

#red potato
model=Sequential()

model.add(LSTM(128,input_shape=(None,1),activation="relu"))

model.add(Dense(1))

model.compile(loss="mean_squared_error",optimizer="adam")

#early stopping and model checking

#Early Stopping
earlystop = EarlyStopping(monitor = 'val_loss', #value being monitored for improvement
                          min_delta = 0.001,         #Abs value and is the main change required before we stop
                          mode='auto',
                          patience = 6,             #no of epocs we wait before stopping
                          verbose = 1,
                          restore_best_weights = True) #keep the best weigts once stopped
#Model Checkpoint
checkpoint = ModelCheckpoint(monitor = "val_loss",
                             mode ='auto',
                             filepath = "E:/cryptics/dataset/best_model_cauli.keras",
                             verbose = 1,
                             save_best_only = True,
                             )

#Reduce learning Rate

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = 0.01,
                              patience = 6,
                              verbose = 1,
                              min_delta = 0.001)


# we put our call backs into a callback list
callback = [earlystop, checkpoint,reduce_lr]

history = model.fit(X_train,y_train,validation_data=(X_test,y_test),callbacks=callback,epochs=220,batch_size=32,verbose=1)

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(loss))



### Lets Do the prediction and check performance metrics
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
train_predict.shape, test_predict.shape

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1))
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1))


# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Train data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

print("Train data explained variance regression score:",
      explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:",
      explained_variance_score(original_ytest, test_predict))


print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

print("Train data MGD: ", mean_gamma_deviance(original_ytrain, train_predict))
print("Test data MGD: ", mean_gamma_deviance(original_ytest, test_predict))
print("----------------------------------------------------------------------")
print("Train data MPD: ", mean_poisson_deviance(original_ytrain, train_predict))
print("Test data MPD: ", mean_poisson_deviance(original_ytest, test_predict))

import seaborn as sns

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)


x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = 30    # Prediction for next 30 days

while(i<pred_days):

    if(len(temp_input)>time_step):

        x_input=np.array(temp_input[1:])
        #print("{} day input {}".format(i,x_input))
        x_input = x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))

        yhat = model.predict(x_input, verbose=0)
        #print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)

        lst_output.extend(yhat.tolist())
        i=i+1

    else:

        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        temp_input.extend(yhat[0].tolist())

        lst_output.extend(yhat.tolist())
        i=i+1

print("Output of predicted next days: ", len(lst_output))

last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
print(last_days)
print(day_pred)

temp_mat = np.empty((len(last_days)+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_days_value = temp_mat
next_predicted_days_value = temp_mat

last_original_days_value[0:time_step+1] = scaler.inverse_transform(closedf[len(closedf)-time_step:]).reshape(1,-1).tolist()[0]
next_predicted_days_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

new_pred_plot = pd.DataFrame({
    'last_original_days_value':last_original_days_value,
    'next_predicted_days_value':next_predicted_days_value
})

names = cycle(['Last 15 days close price','Predicted next 30 days close price'])


# Assuming you have new_pred_plot DataFrame with necessary columns



lstmdf=closedf.tolist()
lstmdf.extend((np.array(lst_output).reshape(-1,1)).tolist())
lstmdf=scaler.inverse_transform(lstmdf).reshape(1,-1).tolist()[0]



# Assuming you have lstmdf list of stock prices





print(next_predicted_days_value[:30])
print(len(next_predicted_days_value))
current2 = df.iloc[len(df)-1]
yesterday2 = df.iloc[len(df)-2]

predictedprices2=next_predicted_days_value[:30]

Shape of close dataframe: (3611, 2)
(3611, 1)
Epoch 1/220


c:\Users\itssh\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



76/78 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0154
Epoch 1: val_loss improved from inf to 0.00521, saving model to E:/cryptics/dataset/best_model_cauli.keras
78/78 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - loss: 0.0151 - val_loss: 0.0052 - learning_rate: 0.0010
Epoch 2/220
77/78 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0028
Epoch 2: val_loss improved from 0.00521 to 0.00478, saving model to E:/cryptics/dataset/best_model_cauli.keras
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0028 - val_loss: 0.0048 - learning_rate: 0.0010
Epoch 3/220
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0024
Epoch 3: val_loss improved from 0.00478 to 0.00421, saving model to E:/cryptics/dataset/best_model_cauli.keras
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0024 - val_loss: 0.0042 - learning_rate: 0.0010
Epoch 4/220
77/78 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0020
Epoch 4: val_loss improved from 0.00421 to 0.00392, saving model to E:/cryptics/dataset/best_model_cauli.keras
78/78 ━━━━

In [7]:
df=pd.read_csv('E:/cryptics/dataset/Tomato_Big.csv')
df
closedf = df[['sn','price']]
print("Shape of close dataframe:", closedf.shape)
del closedf['sn']
scaler=MinMaxScaler(feature_range=(0,1))
closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))
print(closedf.shape)

training_size=int(len(closedf)*0.70)
test_size=len(closedf)-training_size
train_data,test_data=closedf[0:training_size,:],closedf[training_size:len(closedf),:1]


def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)
time_step = 30
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)


# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

#red potato
model=Sequential()

model.add(LSTM(128,input_shape=(None,1),activation="relu"))

model.add(Dense(1))

model.compile(loss="mean_squared_error",optimizer="adam")

#early stopping and model checking

#Early Stopping
earlystop = EarlyStopping(monitor = 'val_loss', #value being monitored for improvement
                          min_delta = 0.001,         #Abs value and is the main change required before we stop
                          mode='auto',
                          patience = 6,             #no of epocs we wait before stopping
                          verbose = 1,
                          restore_best_weights = True) #keep the best weigts once stopped
#Model Checkpoint
checkpoint = ModelCheckpoint(monitor = "val_loss",
                             mode ='auto',
                             filepath = "E:/cryptics/dataset/best_model_tomato.keras",
                             verbose = 1,
                             save_best_only = True,
                             )

#Reduce learning Rate

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = 0.01,
                              patience = 6,
                              verbose = 1,
                              min_delta = 0.001)


# we put our call backs into a callback list
callback = [earlystop, checkpoint,reduce_lr]

history = model.fit(X_train,y_train,validation_data=(X_test,y_test),callbacks=callback,epochs=220,batch_size=32,verbose=1)

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(loss))



### Lets Do the prediction and check performance metrics
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
train_predict.shape, test_predict.shape

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1))
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1))


# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Train data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

print("Train data explained variance regression score:",
      explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:",
      explained_variance_score(original_ytest, test_predict))


print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

print("Train data MGD: ", mean_gamma_deviance(original_ytrain, train_predict))
print("Test data MGD: ", mean_gamma_deviance(original_ytest, test_predict))
print("----------------------------------------------------------------------")
print("Train data MPD: ", mean_poisson_deviance(original_ytrain, train_predict))
print("Test data MPD: ", mean_poisson_deviance(original_ytest, test_predict))

import seaborn as sns

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)


x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = 30    # Prediction for next 30 days

while(i<pred_days):

    if(len(temp_input)>time_step):

        x_input=np.array(temp_input[1:])
        #print("{} day input {}".format(i,x_input))
        x_input = x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))

        yhat = model.predict(x_input, verbose=0)
        #print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)

        lst_output.extend(yhat.tolist())
        i=i+1

    else:

        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        temp_input.extend(yhat[0].tolist())

        lst_output.extend(yhat.tolist())
        i=i+1

print("Output of predicted next days: ", len(lst_output))

last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
print(last_days)
print(day_pred)

temp_mat = np.empty((len(last_days)+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_days_value = temp_mat
next_predicted_days_value = temp_mat

last_original_days_value[0:time_step+1] = scaler.inverse_transform(closedf[len(closedf)-time_step:]).reshape(1,-1).tolist()[0]
next_predicted_days_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

new_pred_plot = pd.DataFrame({
    'last_original_days_value':last_original_days_value,
    'next_predicted_days_value':next_predicted_days_value
})

names = cycle(['Last 15 days close price','Predicted next 30 days close price'])


# Assuming you have new_pred_plot DataFrame with necessary columns



lstmdf=closedf.tolist()
lstmdf.extend((np.array(lst_output).reshape(-1,1)).tolist())
lstmdf=scaler.inverse_transform(lstmdf).reshape(1,-1).tolist()[0]



# Assuming you have lstmdf list of stock prices





print(next_predicted_days_value[:30])
print(len(next_predicted_days_value))
current3 =df.iloc[len(df)-1]
yesterday3 =df.iloc[len(df)-2]

predictedprices3=next_predicted_days_value[:30]

Shape of close dataframe: (3159, 2)
(3159, 1)
Epoch 1/220


c:\Users\itssh\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



65/69 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0190
Epoch 1: val_loss improved from inf to 0.00545, saving model to E:/cryptics/dataset/best_model_tomato.keras
69/69 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - loss: 0.0182 - val_loss: 0.0054 - learning_rate: 0.0010
Epoch 2/220
67/69 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0028
Epoch 2: val_loss improved from 0.00545 to 0.00474, saving model to E:/cryptics/dataset/best_model_tomato.keras
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0028 - val_loss: 0.0047 - learning_rate: 0.0010
Epoch 3/220
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0025
Epoch 3: val_loss improved from 0.00474 to 0.00380, saving model to E:/cryptics/dataset/best_model_tomato.keras
69/69 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0025 - val_loss: 0.0038 - learning_rate: 0.0010
Epoch 4/220
67/69 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0023
Epoch 4: val_loss did not improve from 0.00380
69/69 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - loss: 0.0023 - val_loss: 0.00

In [9]:
from flask import Flask, jsonify
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

print(predictedprices)
print(predictedprices1)

@app.route('/getPotatoValues', methods=['GET'])
def get_potato_values():
    # Ensure there are at least 3 values in the predicted_prices array
    if len(predictedprices) >= 3:
        potato_data = {
            "name": "potato",
            "currentPrice": math.ceil(current[-1]),
            "yesterdayPrice":math.ceil( yesterday[-1]),
            "tomorrowPrice": math.ceil(predictedprices[0]),
            "description": "This is potato",
            "locality": "Nepali",
        }
        
        banana_data = {
            "name": "banana",
            "currentPrice": math.ceil(current1[-1]),
            "yesterdayPrice": math.ceil(yesterday1[-1]),
            "tomorrowPrice": math.ceil(predictedprices1[0]),
            "description": "This is Banana",
            "locality": "Indian",
        }
        cauli_data = {
            "name": "Cauli",
            "currentPrice": math.ceil(current2[-1]),
            "yesterdayPrice":math.ceil( yesterday2[-1]),
            "tomorrowPrice": math.ceil(predictedprices2[0]),
            "description": "This is Local Cauli",
            "locality": "Nepali",
        }
        tomato_data = {
            "name": "Big Tomato",
            "currentPrice":math.ceil( current3[-1]),
            "yesterdayPrice":math.ceil( yesterday3[-1]),
            "tomorrowPrice": math.ceil(predictedprices3[0]),
            "description": "This is Local Hybrid Tomato",
            "locality": "Nepali",
        }
        return jsonify(potato_data,banana_data,cauli_data,tomato_data)
    else:
        return jsonify({"error": "Not enough data available"}), 404



    
if __name__ == '__main__':
    app.run(port=27333)

[31.0, 28.999999999999996, 30.0, 30.0, 28.999999999999996, 28.999999999999996, 28.999999999999996, 29.25, 28.999999999999996, 28.999999999999996, 28.999999999999996, 28.999999999999996, 28.999999999999996, 28.999999999999996, 28.999999999999996, 31.0, 31.0, 32.5, 32.0, 32.0, 34.0, 34.0, 33.8, 33.33, 34.0, 33.6, 33.4, 33.5, 33.0, 33.5]
[130.0, 130.0, 130.0, 130.0, 130.0, 133.33, 133.33, 133.33, 133.33, 133.33, 133.33, 133.33, 133.33, 133.33, 130.0, 130.0, 130.0, 130.0, 130.0, 140.0, 145.0, 136.67, 135.0, 136.67, 125.00000000000001, 125.00000000000001, 130.0, 130.0, 130.0, 130.0]
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:27333
Press CTRL+C to quit
C:\Users\itssh\AppData\Local\Temp\ipykernel_7856\3134589406.py:16: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\itssh\AppData\Local\Temp\ipykernel_7856\3134589406.py:17: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\itssh\AppData\Local\Temp\ipykernel_7856\3134589406.py:25: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\itssh\AppData\Local\Temp\ipyke